### Ejercicios

1. Usa la [siguiente](https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Tokio_2020) página web.

2. Extrae la tabla de las medallas olímpicas que tenéis en la página web.

3. Mete todo el código de la extracción de datos en una función.

4. Los resultados deben estar en un *DataFrame* y hacer una exploración básica con los métodos aprendidos en la lección. 

In [106]:
# Importamos librerías necesarias

import pandas as pd
from bs4 import BeautifulSoup
import re 
import requests

pd.set_option('display.max_columns', None)

# Empezamos haciendo una llamada a la url.

url = "https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos_de_Tokio_2020"
respuesta = requests.get(url)

if respuesta.status_code != 200:
    print(f"Ha habido un error durante la llamada por: {respuesta.reason}")

else:           
    sopa = BeautifulSoup(respuesta.content, "html.parser")
    tablas = sopa.find_all("table")
    # print(f"\nHay {len(tablas)} tablas en la página web") #La tabla que nos interesa es la primera.
    lista_encabezados = []      # Empezaremos por capturar los encabezados de la tabla
    encabezados = tablas[0].findAll("th")
    for encabezado in encabezados:
        if len(encabezado.text) > 1:
            lista_encabezados.append(encabezado.text.replace("\n", ""))
        else:
             lista_encabezados.append(encabezado.find("img").get("alt"))
    # print(lista_encabezados)   

    # Ahora extraeremos las filas de la tabla y de ahí sacaremos los datos de ranking, medallas y total.

    filas = sopa.select('tr') 

    lista_numeros = []
    lista_paises = []
    lista_oros = []
    lista_platas = []
    lista_bronces = []
    lista_total = []

    for fila in filas[1:len(filas)-3]:      # La primera fila no la itero porque son los encabezados. Llego hasta Siria.
        celdas = fila.find_all("td")
        lista_numeros.append(int((celdas[0].text)))   # Capturo la posición en el ranking. Paso la cadena a entero.
        lista_paises.append(celdas[1].text.strip())   # Capturo el país y limpio la cadena.
        lista_oros.append(int(celdas[2].text))        # Capturo las medallas y paso la cadena a entero.
        lista_platas.append(int(celdas[3].text))
        lista_bronces.append(int(celdas[4].text))
        lista_total.append(int(celdas[5].text.strip()))
        
    
    medallero = {}

    medallero[lista_encabezados[0]] = lista_numeros
    medallero[lista_encabezados[1]] = lista_paises
    medallero[lista_encabezados[2]] = lista_oros
    medallero[lista_encabezados[3]] = lista_platas
    medallero[lista_encabezados[4]] = lista_bronces
    medallero[lista_encabezados[5]] = lista_total

    df = pd.DataFrame(medallero)
    print(df)



    Núm.                  País  Oro  Plata  Bronce  Total
0      1  Estados Unidos (USA)   39     41      33    113
1      2           China (CHN)   38     32      19     89
2      3           Japón (JPN)   27     14      17     58
3      4     Reino Unido (GBR)   22     20      22     64
4      5           Rusia (RUS)   20     28      23     71
..   ...                   ...  ...    ...     ...    ...
88    86           Ghana (GHA)    0      0       1      1
89    86         Granada (GRN)    0      0       1      1
90    86          Kuwait (KUW)    0      0       1      1
91    86        Moldavia (MDA)    0      0       1      1
92    86           Siria (SYR)    0      0       1      1

[93 rows x 6 columns]


In [103]:
# Exploración básica de los métodos de Dataframe

print(f"El tamaño del dataframe es {df.shape}\n")
print(f"Las primeras 10 posiciones de la tabla son \n{df.head(10)}\n\n")
print(f"Las últimas 3 posiciones de la tabla son \n{df.tail(3)}\n\n")
print(f"Los datos de las columnas son\n\n {df.dtypes}\n\n")
print(f"Un resumen de la tabla sería: \n\n{df.info()}\n\n")

print("\nestadisticos numéricos", df.describe().T)
print()
print("\nestadisticos de todos los valores", df.describe(include = object).T)
print()
print("\ntotal de valores nulos", df.isnull())
print("\ntotal de valores duplicados", df.duplicated())

El tamaño del dataframe es (93, 6)

Las primeras 10 posiciones de la tabla son 
   Núm.                  País  Oro  Plata  Bronce  Total
0     1  Estados Unidos (USA)   39     41      33    113
1     2           China (CHN)   38     32      19     89
2     3           Japón (JPN)   27     14      17     58
3     4     Reino Unido (GBR)   22     20      22     64
4     5           Rusia (RUS)   20     28      23     71
5     6       Australia (AUS)   17      7      22     46
6     7    Países Bajos (NED)   10     12      14     36
7     8         Francia (FRA)   10     12      11     33
8     9        Alemania (GER)   10     11      16     37
9    10          Italia (ITA)   10     10      20     40


Las últimas 3 posiciones de la tabla son 
    Núm.            País  Oro  Plata  Bronce  Total
90    86    Kuwait (KUW)    0      0       1      1
91    86  Moldavia (MDA)    0      0       1      1
92    86     Siria (SYR)    0      0       1      1


Los datos de las columnas son

 Núm.   